## This note Book will demonstrate different configurations and executions of LLM-Analyst

In [2]:
from llm_analyst.core.research_analyst import LLMAnalyst
query = "What happened in the latest burning man floods?"
llm_analyst = LLMAnalyst(query)
agent_details = llm_analyst.choose_agent()



In [3]:
print(llm_analyst.cfg)

internet_search = <function ddgs_search at 0x12a8b45e0>
embedding_provider = client=<openai.resources.embeddings.Embeddings object at 0x12f566290> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x12f589050> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002' openai_api_version='' openai_api_base=None openai_api_type='' openai_proxy='' embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None
llm_provider = <class 'llm_analyst.chat_models.groq.GROQ_Model'>
fast_llm_model = gpt-3.5-turbo-16k
smart_llm_model = llama3-70b-8192
fast_token_limit = 2000
smart_token_limit = 4000
br

In [4]:
await llm_analyst.conduct_research()
llm_analyst.context


/Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


['Source: https://www.cnn.com/2023/09/05/us/burning-man-storms-shelter-exodus-tuesday/index.html\nTitle: \nContent: Matt Mills McKnight/Reuters\n    Why did the rain have such an impact on Burning Man?\n            After the first rains soaked campsites Friday, event leaders alerted attendees that driving was no longer allowed in the city. They halted entry and exit in the area, urged people to “shelter in place,” and conserve food and water.\n    \n            The remote area in northwest Nevada was hit with up to 0.8 inches – about twice the average September rainfall – in just 24 hours between Friday and Saturday morning. And more rain followed Sunday. \n    \nAmerican DJ Diplo performs during the 2023 Governors Ball Music Festival at Flushing Meadows Corona Park in New York City, on June 9, 2023.\nAngela Weiss/AFP/Getty Images\nDiplo hitchhiked ride out of rain-drenched Burning Man after walking miles \'through the mud\' and actually made it to his DC concert\n\nSource: https://www

In [5]:
from llm_analyst.core.research_writer import LLMWriter
report = await llm_analyst.write_report()
llm_writer = LLMWriter(query)
llm_writer.report_md = report
await llm_writer.write_to_pdf()

Report written to /Users/dan/llm_analyst_out/f0f472ed971a460590d06e60fba2acf3.pdf


'/Users/dan/llm_analyst_out/f0f472ed971a460590d06e60fba2acf3.pdf'

In [6]:
total_len=0
for line in llm_analyst.context:
    total_len += len(line)
    
total_len


31515